In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats as st
import math as mt
import calendar
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import locale

In [2]:
#Datos 
datos_cumple_y_oraciones = pd.read_csv("Cumpleaños Misión Cumbres - Respuestas de formulario 1.csv")
#Formato de columnas 
col_arregladas = []
for col in datos_cumple_y_oraciones.columns: 
    col_arregladas.append(col.strip().lower().replace(" ", "_"))
    
datos_cumple_y_oraciones.columns = col_arregladas

datos_cumple_y_oraciones.info()
display(datos_cumple_y_oraciones.head())
#cumpleaños iglesia. nueva columna tomando mes y dia
datos_cumple_y_oraciones.loc[datos_cumple_y_oraciones["nombre_completo"] == "Nelly Saicedo Cabrera", "fecha_de_nacimiento"] = "01/08/1963" #tenía que nació en año 63 complicado.
#Extraer día y més
datos_cumple_y_oraciones["fecha_de_nacimiento"] = pd.to_datetime(datos_cumple_y_oraciones["fecha_de_nacimiento"], format="%d/%m/%Y")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 4 columns):
 #   Column                                                                   Non-Null Count  Dtype 
---  ------                                                                   --------------  ----- 
 0   marca_temporal                                                           17 non-null     object
 1   nombre_completo                                                          17 non-null     object
 2   fecha_de_nacimiento                                                      17 non-null     object
 3   ¿hay_alguna_petición_de_oración_que_te_gustaría_compartir_con_nosotros?  9 non-null      object
dtypes: object(4)
memory usage: 676.0+ bytes


,marca_temporal,nombre_completo,fecha_de_nacimiento,¿hay_alguna_petición_de_oración_que_te_gustaría_compartir_con_nosotros?
0,20/01/2025 18:08:45,David Sánchez,1/05/1989,Por gracia y sabiduría :)
1,20/01/2025 18:17:01,David Humberto Suárez Garza,1/06/1999,Salud financiera y mejorar sociabilidad para s...
2,20/01/2025 18:20:20,Angela Gonzalez Santos,11/10/1999,NaN
3,20/01/2025 18:24:02,Juan Marcos Sanchez,1/03/2025,NaN
4,20/01/2025 18:30:23,Susan Daniela Cordero Ramos,19/03/2007,Mi semestre en la Facultad


In [3]:
# Extraer día y mes
datos_cumple_y_oraciones["día"] = datos_cumple_y_oraciones["fecha_de_nacimiento"].dt.day
datos_cumple_y_oraciones["mes"] = datos_cumple_y_oraciones["fecha_de_nacimiento"].dt.month

# Ordenar por mes y día
cumples_ordenados = datos_cumple_y_oraciones.sort_values(["mes", "día"])

In [4]:
# Función para crear un calendario mensual con cumpleaños


#def crear_calendario(mes, datos, año=2025, max_caracteres=20, font_size=7, font_family="Comic Sans MS"):
#    """
#    Crear un calendario mensual con los días de cumpleaños bien organizados.
#    
#    Parámetros:
#    - mes: Número del mes (1-12).
#    - datos: DataFrame con columnas 'mes', 'día' y 'nombre_completo'.
#    - año: Año para el calendario (por defecto, 2025).
#    - max_caracteres: Máximo número de caracteres por línea para nombres largos.
#    - font_size: Tamaño de fuente para el texto.
#    - font_family: Familia de fuente para el texto (opciones como "Comic Sans MS", "Arial Rounded", etc.).
#    """
#    def dividir_nombres(nombres):
#        """Divide nombres largos en varias líneas."""
#        texto_dividido = []
#        linea_actual = ""
#        for nombre in nombres:
#            if len(linea_actual) + len(nombre) + 2 > max_caracteres:  # +2 por la coma y espacio
#                texto_dividido.append(linea_actual)
#                linea_actual = nombre
#            else:
#                linea_actual += (", " if linea_actual else "") + nombre
#        texto_dividido.append(linea_actual)  # Agregar la última línea
#        return "\n".join(texto_dividido)
#    
#    # Filtrar datos por mes
#    cumple_mes = datos[datos["mes"] == mes]
#    
#    # Crear el calendario para el mes y año dados
#    cal = calendar.monthcalendar(año, mes)
#    fig, ax = plt.subplots(figsize=(10, 6))
#    ax.axis("off")
#    
#    # Título del mes (reducido para ocupar menos espacio)
#    ax.set_title(f"{calendar.month_name[mes]} {año}", fontsize=16, weight="bold", color="navy", family=font_family, pad=5)
#    
#    # Nombres de los días de la semana (de domingo a sábado)
#    dias = ["Dom", "Lun", "Mar", "Mié", "Jue", "Vie", "Sáb"]
#    for i, dia in enumerate(dias):
#        ax.text(i, 0.1, dia, ha="center", va="center", fontsize=10, fontweight="bold", family=font_family)
#    
#    # Dibujar el calendario
#    for i, week in enumerate(cal):  # Cada semana
#        for j, day in enumerate(week):  # Cada día
#            if day != 0:  # Días válidos
#                cumple = cumple_mes[cumple_mes["día"] == day]
#                nombres = cumple["nombre_completo"].tolist()
#                texto = f"{day}\n" + dividir_nombres(nombres)
#                
#                # Ajustar el cuadro para evitar empalmes y dar espacio
#                ax.text(
#                    j, -i - 1, texto,
#                    ha="center", va="top", fontsize=font_size, family=font_family,
#                    bbox=dict(boxstyle="round,pad=0.5", edgecolor="black", facecolor="white")
#                )
#
#    # Configuración de límites (ajustamos el espacio)
#    ax.set_xlim(-0.5, 6.5)
#    ax.set_ylim(-len(cal), 1)
#    plt.subplots_adjust(top=0.85, bottom=0.1, left=0.1, right=0.9)
#    return fig
#
## Función para crear el calendario completo del año en PDF
#def crear_calendario_anual(datos, año=2025):
#    with PdfPages(f"calendario_{año}.pdf") as pdf:
#        for mes in range(1, 13):  # Para los 12 meses
#            fig = crear_calendario(mes, datos, año)
#            pdf.savefig(fig)  # Guardar cada mes en el PDF
#            plt.close(fig)  # Cerrar la figura para liberar memoria

In [5]:
# Crear calendario completo
#crear_calendario_anual(datos_cumple_y_oraciones)

In [6]:
#lista sintetizada
def crear_lista_sintetizada(datos, año=2025, font_family="Comic Sans MS", font_size=10):
    """
    Genera un PDF con una lista sintetizada de cumpleaños en una sola página, con colores distintos por mes.
    
    Parámetros:
    - datos: DataFrame con columnas 'mes', 'día' y 'nombre_completo'.
    - año: Año del calendario (por defecto, 2025).
    - font_family: Familia de fuente para el texto.
    - font_size: Tamaño de fuente para el texto.
    """
    # Configurar el idioma para nombres en español
    locale.setlocale(locale.LC_TIME, "es_ES.utf8")  # Cambia según el sistema si no funciona
    
    # Ordenar datos por mes y día
    datos = datos.sort_values(by=["mes", "día"])
    
    # Obtener nombres de los meses y días de la semana
    meses = {i: calendar.month_name[i].capitalize() for i in range(1, 13)}
    dias_semana = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]  # Manualmente corregido
    
    # Colores asignados a los meses (más vivos)
    colores_meses = {
        1: "#FF6F61", 2: "#F06292", 3: "#BA68C8", 4: "#9575CD", 5: "#7986CB",
        6: "#64B5F6", 7: "#4FC3F7", 8: "#4DD0E1", 9: "#4DB6AC", 10: "#81C784",
        11: "#AED581", 12: "#FFD54F"
    }
    
    # Crear el PDF
    with PdfPages(f"calendario_sintetizado_{año}.pdf") as pdf:
        fig, ax = plt.subplots(figsize=(8.5, 11))  # Tamaño carta
        ax.axis("off")
        
        # Título
        ax.text(
            0.5, 0.95, f"Calendario de Cumpleaños {año}", 
            fontsize=16, fontweight="bold", ha="center", family=font_family, color="#1565C0"
        )
        
        # Generar lista de cumpleaños
        y_pos = 0.9
        line_spacing = 0.03  # Ajustar el espaciado entre líneas
        mes_actual = None
        
        for _, row in datos.iterrows():
            dia_mes = row["día"]
            dia_semana = dias_semana[calendar.weekday(año, row["mes"], dia_mes)]
            mes = meses[row["mes"]]
            nombres = row["nombre_completo"]
            
            # Separador si cambia de mes
            if mes_actual != row["mes"]:
                mes_actual = row["mes"]
                y_pos -= 0.04  # Separar un poco más los títulos de los meses
                ax.text(
                    0.5, y_pos, f"--- {mes.upper()} ---", 
                    fontsize=12, fontweight="bold", ha="center", family=font_family, color=colores_meses[mes_actual]
                )
                y_pos -= line_spacing
            
            # Agregar cumpleaños
            ax.text(
                0.05, y_pos, f"{dia_mes} ({dia_semana}): {nombres}", 
                fontsize=font_size, family=font_family, va="top", color="black"
            )
            y_pos -= line_spacing
            if y_pos < 0.05:  # Si se llena la página
                pdf.savefig(fig)
                plt.close(fig)
                fig, ax = plt.subplots(figsize=(8.5, 11))
                ax.axis("off")
                y_pos = 0.9  # Reiniciar posición
                
        # Guardar la última página
        pdf.savefig(fig)
        plt.close(fig)

# Uso de la función
crear_lista_sintetizada(datos_cumple_y_oraciones) #Calendario realizado

#Visualización de oraciones. Qué hacer? 
-Pay